In [3]:
import sys, os
src_path = os.path.abspath("../src")
if src_path not in sys.path:
    sys.path.append(src_path)
                           

In [4]:
from utils.notebook_helper import setup_notebook_environment
setup_notebook_environment()

'Environment ready for experiments'

In [5]:
from llm.router import LLMRouter

client = LLMRouter(model="gpt4")


In [6]:
print(client.ask("Hello, who are you?"))

Hello! I'm an AI language model here to assist you with any questions or tasks you have. How can I help you today?


In [ ]:
import requests
response = requests.get('http://langchain_api:8000/health')
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")

In [65]:
import fitz  # PyMuPDF
import re

def extract_text_from_pdf(pdf_path):
    """Extract full text from a PDF file."""
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    return full_text

def chunk_by_paragraph_tags(text):
    """
    Chunk text using [XXXX] tags like [0001], [0002]...
    Returns list of (tag, paragraph_text) tuples.
    """
    pattern = re.compile(r"(\[\d{4}\])")
    parts = pattern.split(text)

    chunks = []
    current_tag = None
    for part in parts:
        if pattern.match(part):
            current_tag = part
        elif current_tag:
            clean_paragraph = part.strip()
            if clean_paragraph:
                chunks.append((current_tag, clean_paragraph))
                current_tag = None
    return chunks

# === Example Usage ===

pdf_path = "EP18823397W1B9.pdf"
text = extract_text_from_pdf(pdf_path)
chunks = chunk_by_paragraph_tags(text)

# Print first 5 chunks
for tag, para in chunks[:100]:
    print(f" Para :: {tag}: {para}...\n")


 Para :: [0001]: The present invention relates to the field of pharmaceuticals, and in particular to phosphorus-containing
compound, preparation method thereof, and use for treating dry eye.
Technical background...

 Para :: [0002]: Tears provide long-lasting moisturization and lubrication to the eyes, which is the key to maintaining vision and
eye comfort. Tears are composed of water, lipids, mucus, antibodies, and specific proteins with anti-infective properties.
These components are secreted by specific glands located around the eyes. When there is an imbalance in the tear
system, people will feel dry eyes....

 Para :: [0003]: Dry eye syndrome is a common ocular surface inflammatory disease. People with dry eye may experience eye
pain, photosensitivity, itching, redness and blurred vision. Dry eye syndrome is caused by multiple inducing factors,
including age, gender, environment, medicine, surgery, and systemic diseases such as autoimmune diseases, diabetes,
thyroid disease, and l

In [55]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 21.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [62]:
import fitz  # PyMuPDF

def extract_pdf_structure(path):
    doc = fitz.open(path)
    structured_text = []
    
    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if "lines" not in b:
                continue
            for l in b["lines"]:
                line_text = ""
                max_font_size = 0
                for s in l["spans"]:
                    line_text += s["text"]
                    if s["size"] > max_font_size:
                        max_font_size = s["size"]
                
                # Heuristic: Assume headings have font size above a threshold
                if max_font_size > 15:  # You can adjust this based on your PDF
                    structured_text.append({
                        "type": "heading",
                        "text": line_text.strip()
                    })
                else:
                    structured_text.append({
                        "type": "paragraph",
                        "text": line_text.strip()
                    })
    
    return structured_text


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte

In [60]:
pdf_path = "EP18823397W1B9.pdf"
content = extract_pdf_structure(pdf_path)

for block in content:
    print(f"[{block['type'].upper()}] {block['text']}")

[PARAGRAPH] (19)
[HEADING] *EP003647315B9*
[PARAGRAPH] (11)
[HEADING] EP 3 647 315 B9
[PARAGRAPH] (12)
[PARAGRAPH] CORRECTED EUROPEAN PATENT SPECIFICATION
[PARAGRAPH] (15) Correction information:
[PARAGRAPH] Corrected version no
[PARAGRAPH] 1
[PARAGRAPH] (W1 B1)
[PARAGRAPH] Corrections, see
[PARAGRAPH] Claims DE
[PARAGRAPH] 4
[PARAGRAPH] Claims EN
[PARAGRAPH] 4
[PARAGRAPH] Claims FR
[PARAGRAPH] 4
[PARAGRAPH] (48) Corrigendum issued on:
[PARAGRAPH] 04.06.2025
[PARAGRAPH] Bulletin 2025/23
[PARAGRAPH] (45) Date of publication and mention
[PARAGRAPH] of the grant of the patent:
[PARAGRAPH] 20.11.2024
[PARAGRAPH] Bulletin 2024/47
[PARAGRAPH] (21) Application number: 18823397.7
[PARAGRAPH] (22) Date of filing: 21.05.2018
[PARAGRAPH] (51) International Patent Classification (IPC):
[PARAGRAPH] A61K 9/00 (2006.01)
[PARAGRAPH] A61K 9/08 (2006.01)
[PARAGRAPH] A61K 47/02 (2006.01)
[PARAGRAPH] C07F 9/53 (2006.01)
[PARAGRAPH] C07F 9/572 (2006.01)
[PARAGRAPH] C07F 9/6503 (2006.01)
[PARAGRAPH] C07F 9/

In [ ]:
from bs4 import BeautifulSoup

with open("EP18823397W1B9.xml", "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "xml")
    plain_text = soup.get_text(separator="\n") 
plain_text

In [ ]:
print("x")

In [72]:
from textwrap import wrap

def chunk_text_blocks(blocks, max_chars=10000):
    chunks = []
    current_chunk = ""
    for block in blocks:
        entry = f"[{block['type'].upper()}] {block['text']}\n\n"
        if len(current_chunk) + len(entry) > max_chars:
            chunks.append(current_chunk)
            current_chunk = entry
        else:
            current_chunk += entry
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

# Split the PDF content into manageable chunks
chunks = chunk_text_blocks(plain_text, max_chars=8000)

In [ ]:
for i, chunk in enumerate(chunks):
    prompt = f"""
You are an expert in chemical document structuring and JSON formatting.

Below is a section of text extracted from a pharmaceutical patent or research paper. Your task is to extract and organize the information into **structured, valid JSON**, following these rules:

---

1. Each top-level section should be a JSON object under one of the following keys:
   - `metadata`: Contains fields like `title`, `inventors`, `application_date`, `publication_date`, `abstract`.
   - `paragraphs`: A list of objects with:
     - `para_number` (if available),
     - `heading` (e.g., "Field of the Invention", "Background", etc.),
     - `text` (full paragraph content).
   - `experiments`: A list of objects, each with:
     - `step_number`, `temperature`, `time`, `quenching`, `extraction`, `product`, `molar_ratio`, `yield`, `mz`, `nmr`, etc., if available.
   - `examples`: A list of named experiments (e.g., "Example 1", "Example 2") with associated `steps`.

2. If the paragraph contains NMR or MS data, extract them under keys `"nmr"` or `"mz"` with appropriate subfields.

3. Keep numerical units as-is. Don’t paraphrase or explain.

4. If data is missing, use `null`.

---

Now format the following section as clean and valid JSON:

[PASTE TEXT CHUNK HERE]


{chunk}

Respond with **JSON only**. No explanation.
"""
    response = client.ask(prompt)
    try:
        results.extend(json.loads(response))
    except Exception as e:
        print(f"Chunk {i} failed: {e}")
        print("Raw output:", response)


Chunk 0 failed: name 'results' is not defined
Raw output:  {
  "metadata": {
    "title": "EP003647315B9",
    "inventors": [
      {
        "name": "SHEN, Wang",
        "location": "Shanghai 201203 (CN)"
      },
      {
        "name": "DING, Yue",
        "location": "Shanghai 201203 (CN)"
      },
      {
        "name": "JIANG, Hao",
        "location": "Shanghai 201203 (CN)"
      },
      {
        "name": "CHEN, Fu li",
        "location": "Shanghai 201203 (CN)"
      },
      {
        "name": "WANG, Jiangfeng",
        "location": "Shanghai 201203 (CN)"
      },
      {
        "name": "WU, Xinglong",
        "location": "Shanghai 201203 (CN)"
      },
      {
        "name": "LI, Cunfei",
        "location": "Shanghai 201203 (CN)"
      },
      {
        "name": "YANG, Liguo",
        "location": "Shanghai 201203 (CN)"
      },
      {
        "name": "HU, Biao",
        "location": "Shanghai 201203 (CN)"
      },
      {
        "name": "JIANG, Qiyang",
        "location

In [ ]:
print(json.dumps(results, indent=2))

In [71]:
# Step 1: Extract raw PDF content
pdf_path = "EP18823397W1B9.pdf"
content = extract_pdf_structure(pdf_path)  # your function

# Step 2: Format this content using an LLM

text = "\n\n".join(f"[{block['type'].upper()}] {block['text']}" for block in content)

prompt = f"""
I have extracted the following content from a PDF. Please convert it into a structured JSON format where each block is clearly identified with its type and content. Here is the text:

{text}
"""

response = client.ask(prompt)
# Step 3: Use/print/save the formatted JSON
print(response)


BadRequestError: Error code: 400 - {'error': {'message': 'This endpoint\'s maximum context length is 32768 tokens. However, you requested about 50709 tokens (50709 of text input). Please reduce the length of either one, or use the "middle-out" transform to compress your prompt automatically.', 'code': 400, 'metadata': {'provider_name': None}}}

In [1]:
print("x")

x


In [12]:
import json
from bs4 import BeautifulSoup

# Function to split text into chunks
def split_text_into_chunks(text, max_chars=8000):
    """Split text into manageable chunks."""
    chunks = []
    current_chunk = ""
    for line in text.splitlines():
        if len(current_chunk) + len(line) + 1 > max_chars:
            chunks.append(current_chunk)
            current_chunk = line
        else:
            current_chunk += "\n" + line
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

# Function to process a single chunk with OpenAI client
def process_chunk_with_openai(client, chunk):
    """Send a chunk to OpenAI client and get headings/subheadings."""
    prompt = f"""
You are an expert in document structuring. Extract headings and subheadings from the following text and return them in JSON format. 
If the text starts mid-paragraph, handle it gracefully. If there are no headings, return an empty JSON array.

Text:
{chunk}

Respond with JSON only.
"""
    response = client.ask(prompt)
    try:
        return json.loads(response)
    except json.JSONDecodeError:
        print("Failed to parse response:", response)
        return []

# Main function to process the XML file
def extract_headings_from_xml(xml_path, client, output_json_path):
    """Extract headings and subheadings from an XML file using OpenAI client."""
    # Read and parse the XML file
    with open("EP18823397W1B9.xml", "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "lxml-xml")  # Explicitly use the lxml XML parser
        text = soup.get_text(separator="\n") 

    # Split text into chunks
    chunks = split_text_into_chunks(text)

    # Process each chunk and collect results
    all_headings = []
    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i + 1}/{len(chunks)}...")
        headings = process_chunk_with_openai(client, chunk)
        all_headings.extend(headings)

    # Save combined headings to a JSON file
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(all_headings, f, indent=2, ensure_ascii=False)

    print(f"Headings saved to {output_json_path}")


# Path to your XML file
xml_path = "EP18823397W1B9.xml"

# Output JSON file
output_json_path = "headings.json"

# Extract headings
extract_headings_from_xml(xml_path, client, output_json_path)

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml-xml. Do you need to install a parser library?

In [5]:
import fitz  # PyMuPDF
import re
import json

def extract_pdf_structure(pdf_path):
    doc = fitz.open(pdf_path)
    metadata = doc.metadata

    # Extract all text
    full_text = ""
    for page in doc:
        full_text += page.get_text("text")

    # Flexible header pattern for patents (all-caps, numbers, common sections)
    header_pattern = re.compile(
        r'^(?:[A-Z][A-Z0-9\s\-]{2,}|[A-Z][a-z]+(?:\s[A-Z][a-z]+)*):?\s*$', re.MULTILINE
    )
    matches = list(header_pattern.finditer(full_text))

    sections = []
    for i, match in enumerate(matches):
        header = match.group().strip(":").strip()
        start = match.end()
        end = matches[i+1].start() if i+1 < len(matches) else len(full_text)
        text = full_text[start:end].strip()
        # Split into paragraphs (double newlines or line breaks)
        paragraphs = [p.strip() for p in re.split(r'\n{2,}', text) if p.strip()]
        sections.append({"header": header, "paragraphs": paragraphs})

    if not sections:
        sections = [{"header": "Full Text", "paragraphs": [full_text.strip()]}]

    result = {
        "metadata": metadata,
        "sections": sections
    }
    return result

# Usage
pdf_path = "EP18823397W1B9.pdf"
structured_json = extract_pdf_structure(pdf_path)
with open("structured_patent.json", "w") as f:
    json.dump(structured_json, f, indent=2)

In [9]:
import xml.etree.ElementTree as ET
import json

def parse_patent_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Auto-detect namespace if present
    ns = {}
    if '}' in root.tag:
        uri = root.tag.split('}')[0].strip('{')
        ns = {'ns': uri}
        def tag(t): return f"{{{uri}}}{t}"
    else:
        def tag(t): return t

    # Extract metadata
    metadata = {}
    for k in ['doc-number', 'country', 'kind', 'date-publ', 'lang']:
        metadata[k] = root.attrib.get(k, "")

    # Extract abstract
    abstract = ""
    abstract_elem = root.find('.//'+tag('abstract'))
    if abstract_elem is not None:
        abstract = " ".join(
            (p.text or '').strip() for p in abstract_elem.findall('.//'+tag('p'))
        )

    # Extract description: headings and paragraphs
    description = []
    desc_elem = root.find('.//'+tag('description'))
    if desc_elem is not None:
        current_section = None
        for elem in desc_elem:
            if elem.tag.endswith('heading'):
                if current_section:
                    description.append(current_section)
                current_section = {"header": (elem.text or '').strip(), "paragraphs": []}
            elif elem.tag.endswith('p'):
                if current_section is None:
                    current_section = {"header": "", "paragraphs": []}
                if elem.text:
                    current_section["paragraphs"].append(elem.text.strip())
        if current_section:
            description.append(current_section)

    # Build result
    result = {
        "metadata": metadata,
        "abstract": abstract,
        "sections": description
    }
    return result

# Usage
xml_path = "EP18823397W1B9.xml"
structured_data = parse_patent_xml(xml_path)
print(json.dumps(structured_data, indent=2, ensure_ascii=False))

{
  "metadata": {
    "doc-number": "3647315",
    "country": "EP",
    "kind": "B9",
    "date-publ": "20250604",
    "lang": "en"
  },
  "abstract": "",
  "sections": [
    {
      "header": "",
      "paragraphs": [
        "The present invention relates to the field of pharmaceuticals, and in particular to phosphorus-containing compound, preparation method thereof, and use for treating dry eye."
      ]
    },
    {
      "header": "",
      "paragraphs": [
        "Tears provide long-lasting moisturization and lubrication to the eyes, which is the key to maintaining vision and eye comfort. Tears are composed of water, lipids, mucus, antibodies, and specific proteins with anti-infective properties. These components are secreted by specific glands located around the eyes. When there is an imbalance in the tear system, people will feel dry eyes.",
        "Dry eye syndrome is a common ocular surface inflammatory disease. People with dry eye may experience eye pain, photosensitivity, 

In [15]:
import xml.etree.ElementTree as ET
import json

def group_by_heading(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    elements = []
    for elem in root.iter():
        if elem.tag == 'heading' or elem.tag == 'p':
            elements.append(elem)

    grouped = {}
    current_heading = None

    for elem in elements:
        if elem.tag == 'heading':
            current_heading = elem.text.strip() if elem.text else "Untitled"
            if current_heading not in grouped:
                grouped[current_heading] = []
        elif elem.tag == 'p':
            if current_heading is None:
                current_heading = "No Heading"
                grouped[current_heading] = []
            grouped[current_heading].append(elem.text.strip() if elem.text else "")

    return grouped



xml_path = "EP18823397W1B9.xml"  # Update path if needed
grouped = group_by_heading(xml_path)
print(json.dumps(grouped, indent=2, ensure_ascii=False))

{
  "Untitled": [
    "The present invention relates to the field of pharmaceuticals, and in particular to phosphorus-containing compound, preparation method thereof, and use for treating dry eye.",
    "Tears provide long-lasting moisturization and lubrication to the eyes, which is the key to maintaining vision and eye comfort. Tears are composed of water, lipids, mucus, antibodies, and specific proteins with anti-infective properties. These components are secreted by specific glands located around the eyes. When there is an imbalance in the tear system, people will feel dry eyes.",
    "Dry eye syndrome is a common ocular surface inflammatory disease. People with dry eye may experience eye pain, photosensitivity, itching, redness and blurred vision. Dry eye syndrome is caused by multiple inducing factors, including age, gender, environment, medicine, surgery, and systemic diseases such as autoimmune diseases, diabetes, thyroid disease, and lymphoma. If dry eye disease is not diagnose